https://docs.google.com/forms/d/e/1FAIpQLSdI6cWyyHCBkn2h0lUXvZM9iGNX3y1QMRbKT0iSVsrm8Qhx_w/viewform?hr_submission=ChcIrpDc8gMSDwi72JqF0wgSBgiGjdbHJxAB

In [1]:
import pandas as pd
import numpy as np

import urllib.request
import io
import time
import copy


from scipy.io import arff
from sklearn.model_selection import KFold
from collections import Counter

from sklearn.preprocessing import MinMaxScaler
import re
import random

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# Preparação de dados

* extração do dataframe

In [2]:
url = 'http://promise.site.uottawa.ca/SERepository/datasets/kc1.arff'
ftpstream = urllib.request.urlopen(url)
data, meta = arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))
kc1 = pd.DataFrame(data)
kc1 = kc1.sample(frac=1, random_state=20)
kc1.reset_index(drop=True, inplace=True)

# Algoritmos

In [3]:
def  euclidian_distance(x1, x2):
    arr = []
    for i in range(len(x1)-1):
        arr.append((x1[i] - x2[i])** 2)
    return np.sqrt(np.sum(arr))


In [4]:
class KNN:

    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y


    def predict(self, X):
        predicted_labels = []
        for _, row in X.iterrows():
            predicted_labels.append(self.predict_func(row))

        return np.array(predicted_labels)
        
    def predict_func(self, x):
        distances = []
        for _, row in self.X_train.iterrows():
            distances.append(euclidian_distance(x, row))
          
        k_indices = np.argsort(distances)[:self.k]        
        k_nearest_labels = [self.y_train.iloc[i] for i in k_indices]

        unique, counts = np.unique(k_nearest_labels, return_counts=True)
        predicted = unique[counts.argmax()]

        return predicted

In [5]:
def attr_class(df):
    Y = df["defects"]
    X = df.drop(columns=["defects"]) 
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    x_scaled = scaler.fit_transform(X)
    X = pd.DataFrame(x_scaled)
    
    Y = Y.apply(str).str.replace("b|'", '')

    
    return X, Y

In [6]:
def _attr_class(df):
    
    x = []
    y = []
    for row in df:
        x.append(row[:-1])
        y.append(row[-1])

    
    return x,y

### LVQ 1

In [7]:
from sklearn.neighbors import KNeighborsClassifier


In [8]:
class LVQ1:   
    
    def __init__(self, n_prototypes):
        self.n_prototypes  = n_prototypes
        self._epochs        = 1
        self._lrate        = 0.25

    
    def random_prototype(self, train):
        n_records  = train.shape[0]
        n_features = train.shape[1]
        prototypes  = []
        records    = []
        random.seed(10)
        for i in range(self.n_prototypes):
            records.append(random.randint(0, n_records-1))
        
        for r in records:
            prototype = [train.iloc[r][i] for i in range(n_features)]
            prototypes.append(prototype)
        
        return prototypes, records
    

    def trainPrototypes(self,train):
        prototypes = []
        records = []
        aux = train
        for i in range(self.n_prototypes):
            prototypes, records = self.random_prototype(train)
        for i in records:
            aux = aux.drop(aux.loc[aux.index==i].index)
        aux = aux.reset_index(drop=True)
        
        x_train, y_train = attr_class(aux)
        x_proto, y_proto = _attr_class(prototypes)

        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(x_train, y_train)
        for epoch in range(self._epochs):            
            rate  = self._lrate * (1.0 - (epoch / float(self._epochs)))
            predicted_classes = knn.predict(x_proto) 
            real_instances = knn.kneighbors(X=x_proto, n_neighbors=1, return_distance=False)
            for idx in range(len(x_proto)):
                prototype = x_proto[idx] # prototipo (ta sendo alterado o valor)
                proto_class = y_proto[idx] #classe do prototipo 
                
                predicted_class = predicted_classes[idx] #classe prevista pra esse prototipo 
                real_instance_index = real_instances[idx][0] #indice instancia real daquele prototipo 
                real_instance = x_train.iloc[real_instance_index] # instancia real
                
                for p in range(len(prototype)): #passa por cada coluna do prototipo
                    proto_val = prototype[p]
                    real_val  = real_instance[p]
                    error = rate * (real_val - proto_val)
                    if proto_class == predicted_class:
                        prototype[p] += error
                    else:
                        prototype[p] -= error

       
        return pd.DataFrame(prototypes, columns = train.columns)

# lvq1 = LVQ1(10)
# df  = lvq1.trainPrototypes(kc2)
# df 

### LVQ 2.1

In [9]:
class LVQ2_1:   
    
    def __init__(self, n_prototypes, window):
        self.n_prototypes  = n_prototypes
        self._epochs        = 10
        self._lrate        = 0.25
        self._window        = window

    
    def inside(self):
        return (1.0 - self._window) / (1.0 + self._window)
    
    def window(self, d1, d2):
        a  = d1/d2
        b  = d2/d1
        minimum = min(a,b)
        return minimum > self.inside()
    
    def updatePrototypes(self, prototype, real, same_class, rate):
        for idx in range(21):
            proto_val = prototype[idx]
            real_val  = real[idx]
            error = rate * (real_val - proto_val)
            if same_class:
                prototype[idx] += error
            else:
                prototype[idx] -= error
        return prototype
    
    def trainPrototypes(self,train, df_prototypes):
        x_train, y_train = attr_class(train)
        x_proto, y_proto = attr_class(df_prototypes)
        knn = KNeighborsClassifier(n_neighbors=2)
        knn.fit(x_proto, y_proto)
        for epoch in range(self._epochs):
            rate  = self._lrate * (1.0 - (epoch / float(self._epochs)))
            x_neighbors = knn.kneighbors(X=x_train, n_neighbors=2, return_distance=False)
            for idx, _ in x_train.iterrows():
                x_instance = x_train.iloc[idx]
                x_class = y_train.iloc[idx]
                
                #mais próximo
                n1_idx   = x_neighbors[idx][0]
                n1       = x_proto.iloc[n1_idx]
                n1       = n1[:21] ##tá adicionando o indice como uma coluna na iteração seguinte
                n1_class = y_proto[n1_idx]
                n1_dist  = euclidian_distance(n1, x_instance) 
                
                #segundo mais próximo
                n2_idx   = x_neighbors[idx][0]
                n2       = x_proto.iloc[n2_idx]
                n2       = n2[:21]
                n2_class = y_proto[n2_idx]
                n2_dist  = euclidian_distance(n2, x_instance)
                
                insideWindow = self.window(n1_dist, n2_dist)
                if insideWindow or n1_class != n2_class:
                    x_proto.iloc[n1_idx] = self.updatePrototypes(n1, x_instance, n1_class == x_class, rate)
                    x_proto.iloc[n2_idx] = self.updatePrototypes(n2, x_instance, n2_class == x_class, rate)
        prototypes = pd.concat([x_proto, y_proto], axis=1)
        prototypes.columns = kc1.columns
        return prototypes 
    
# lvq2 = LVQ2_1(100, 0.25)
# df2  = lvq2.trainPrototypes(kc2, df)
# df2

### LVQ 3

In [10]:
class LVQ3:   
    
    def __init__(self, n_prototypes, window):
        self.n_prototypes  = n_prototypes
        self._epochs       = 10
        self._lrate        = 0.25
        self._window       = window
        
        
    def getNeighborPrototype(self, prototypes, row):
        distances  = []
        final_dist = []
        for _,proto in prototypes.iterrows():
            dist = euclidian_distance(proto, row)
            distances.append((proto, dist))
        distances.sort(key=lambda tup: tup[1])
        final_dist.append(distances[1])
        final_dist.append(distances[2])
        return final_dist
    
    def inside(self):
        return (1.0 - self._window) / (1.0 + self._window)
    
    def window(self, d1, d2):
        a  = d1/d2
        b  = d2/d1
        minimum = min(a,b)
        return minimum > self.inside()
    
    def updatePrototypes(self, prototype, real, same_class, rate):
        for idx in range(21):
            proto_val = prototype[idx]
            real_val  = real[idx]
            error = rate * (real_val - proto_val)
            if same_class:
                prototype[idx] += error
            else:
                prototype[idx] -= error
        return prototype
    
    def trainPrototypes(self,train, df_prototypes):
        x_train, y_train = attr_class(train)
        x_proto, y_proto = attr_class(df_prototypes)
        knn = KNeighborsClassifier(n_neighbors=2)
        knn.fit(x_proto, y_proto)        
        for epoch in range(self._epochs):
            rate = self._lrate * (1.0 - (epoch / float(self._epochs)))
            x_neighbors = knn.kneighbors(X=x_train, n_neighbors=2, return_distance=False)
            for idx, _ in x_train.iterrows():
                x_instance = x_train.iloc[idx]
                x_class = y_train.iloc[idx]

                #mais próximo
                n1_idx   = x_neighbors[idx][0]
                n1       = x_proto.iloc[n1_idx]
                n1       = n1[:21] ##tá adicionando o indice como uma coluna na iteração seguinte
                n1_class = y_proto[n1_idx]
                n1_dist  = euclidian_distance(n1, x_instance) 

                #segundo mais próximo
                n2_idx   = x_neighbors[idx][0]
                n2       = x_proto.iloc[n2_idx]
                n2       = n2[:21]
                n2_class = y_proto[n2_idx]
                n2_dist  = euclidian_distance(n2, x_instance)

                insideWindow = self.window(n1_dist, n2_dist)
                if insideWindow:
                    if n1_class != n2_class:
                        x_proto.iloc[n1_idx] = self.updatePrototypes(n1, x_instance, n1_class == x_class, rate)
                        x_proto.iloc[n2_idx] = self.updatePrototypes(n2, x_instance, n2_class == x_class, rate)
                    else:
                        x_proto.iloc[n1_idx] = self.updatePrototypes(n1, x_instance, True, rate)
                        x_proto.iloc[n2_idx] = self.updatePrototypes(n2, x_instance, True, rate)

        prototypes = pd.concat([x_proto, y_proto], axis=1)
        prototypes.columns = kc1.columns
        return prototypes
    
# lvq3 = LVQ3(10, 0.3)
# res  = lvq3.trainPrototypes(kc1, df)
# df3

In [11]:
# lvq3 = LVQ3(10, 0.3, 0.5)
# res  = lvq3.trainPrototypes(df)
# df3 = pd.DataFrame(res, columns = kc2.columns)
# df3

## KNN 


In [ ]:
def run(df, title):
    kf = KFold(n_splits=5)
    results = []
    n_proto = [10, 50, 100, 200]
    for n in n_proto:
        for k in [1,3]:
            accuracies_simples = []
            accuracies_lvq1   = []
            accuracies_lvq2   = []
            accuracies_lvq3   = []
            index = 0
            for train, test in kf.split(df):
                #separação conj de teste atributos e classe
                attr, df_class = attr_class(df)

                # KNN com conjunto normal
                knn  = KNN(k)
                knn.fit(attr, df_class)
                predictions_simples = knn.predict(attr.iloc[test])
                acc_simples = (np.sum(predictions_simples == df_class.iloc[test]) / len(test)) * 100
                accuracies_simples.append(acc_simples)
                
                #definição do dataset de LVQ1
                lvq1 = LVQ1(n)
                aux = df.iloc[train]
                aux.reset_index(drop=True, inplace=True)
                res  = lvq1.trainPrototypes(aux)
                lvq1_df  = pd.DataFrame(res, columns = df.columns)
                
                #separação do treinamento LVQ1 atributos e classe
                X, Y = attr_class(lvq1_df)
                
                #KNN com conjunto do LVQ1
                knn  = KNN(k)
                knn.fit(X, Y)
                predictions_lvq1 = knn.predict(attr.iloc[test])
                acc_lvq1 = (np.sum(predictions_lvq1 == df_class.iloc[test]) / len(test)) * 100
                accuracies_lvq1.append(acc_lvq1)
                
                #definição do dataset de LVQ2.1
                lvq2 = LVQ2_1(n, 0.25)
                res  = lvq2.trainPrototypes(aux,lvq1_df)
                lvq2_df  = pd.DataFrame(res, columns = df.columns)
                
                #separação do treinamento LVQ2.1 atributos e classe
                X, Y = attr_class(lvq2_df)
                
                #KNN com conjunto do LVQ2.1
                knn  = KNN(k)
                knn.fit(X, Y)
                predictions_lvq2 = knn.predict(attr.iloc[test])
                acc_lvq2 = (np.sum(predictions_lvq2 == df_class.iloc[test]) / len(test)) * 100
                accuracies_lvq2.append(acc_lvq2)
                
                #definição do dataset de LVQ3
                lvq3 = LVQ3(n, 0.5)
                res  = lvq3.trainPrototypes(aux,lvq1_df)
                lvq3_df  = pd.DataFrame(res, columns = df.columns)
                
                #separação do treinamento LVQ3 atributos e classe
                X, Y = attr_class(lvq3_df)
                
                #KNN com conjunto do LVQ3
                knn  = KNN(k)
                knn.fit(X, Y)
                predictions_lvq3 = knn.predict(attr.iloc[test])
                acc_lvq3 = (np.sum(predictions_lvq3 == df_class.iloc[test]) / len(test)) * 100
                accuracies_lvq3.append(acc_lvq3)
            
            print("({}, {}): simples {} lvq1 {} lvq2 {} lvq3 {}".format(k, n, np.mean(accuracies_simples),np.mean(accuracies_lvq1),np.mean(accuracies_lvq2), np.mean(accuracies_lvq3)))
            temp = {
                'dataframe': title,
                '(k, prototypes)':(k, n),
                'acc_simples': np.mean(accuracies_simples),
                'std_simples': np.std(accuracies_simples),
                'acc_LVQ1'   : np.mean(accuracies_lvq1),
                'std_LVQ1'   : np.std(accuracies_lvq1),
                'acc_LVQ2.1' : np.mean(accuracies_lvq2),
                'std_LVQ2.1' : np.std(accuracies_lvq2),
                'acc_LVQ3'   : np.mean(accuracies_lvq3),
                'std_LVQ3'   : np.std(accuracies_lvq3),
            }
            results.append(temp)
    df = pd.DataFrame(results)
    return df

df = run(kc1, 'KC1')
df

(1, 10): simples 96.25491101079578 lvq1 84.4941518163704 lvq2 84.92080467404396 lvq3 84.35185914827031
(3, 10): simples 90.61149823822765 lvq1 84.54165775461267 lvq2 84.54165775461267 lvq3 84.54165775461267
(1, 50): simples 96.25491101079578 lvq1 82.7863020792291 lvq2 42.56745955803717 lvq3 82.40771802636466
(3, 50): simples 90.61149823822765 lvq1 84.49426438968378 lvq2 71.66698562438788 lvq3 84.44664587812812
(1, 100): simples 96.25491101079578 lvq1 79.5162724724477 lvq2 43.23085409372854 lvq3 81.0336481633664


menos protótipos > resultado mto ruim > desbalanço do conjunto

## Plot

curva roc?

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
def _printComparative(df):
    

    fig = make_subplots(rows=1, cols=1,
    specs=[[{"type": "Scatter"}]],
    subplot_titles=("Acurácia"))

    
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_simples'], name='Simples', mode = 'lines+markers'), col = 1, row = 1) 
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_LVQ1'], name='LVQ1', mode = 'lines+markers'), col = 1, row = 1)
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_LVQ2.1'], name='LVQ2.1', mode = 'lines+markers'), col = 1, row = 1)
    fig.add_trace(go.Scatter(x=df.index, y = df['acc_LVQ3'], name='LVQ3', mode = 'lines+markers'), col = 1, row = 1)
    
    fig.update_layout(height=1000, width=1000)

    return fig


In [ ]:
fig = _printComparative(df)
fig.show()